In [21]:
!pip install git-filter-repo

In [16]:
import pandas as pd
import numpy as np
import ssl
import certifi
import json
import statsmodels.api as sm
from urllib.request import urlopen
from tqdm import tqdm
from sqlalchemy import create_engine


def load_monthly_export_forecast(db_info, table_name='trade_forecast_by_month'):
    """
    수출 예측 데이터 전체를 월별 원형 그대로 불러오는 함수 (root_hs_code 조건 없이 전체)

    Parameters:
    - db_info (dict): {
        'host': DB 주소 (str),
        'port': 포트 번호 (int),
        'user': 사용자명 (str),
        'password': 비밀번호 (str),
        'database': 데이터베이스명 (str)
      }
    - table_name (str): 테이블 이름 (기본값: 'trade_forecast_by_month')

    Returns:
    - pd.DataFrame: 월별 수출 예측 원본 데이터프레임
    """
    try:
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )

        query = f"""
            SELECT *
            FROM {table_name}
            ORDER BY root_hs_code, period
        """

        df = pd.read_sql(query, con=engine)
        df = df.drop_duplicates(subset=['root_hs_code', 'period'])
        df['period'] = pd.to_datetime(df['period'])

        return df

    except Exception as e:
        print(f"❌ 월별 데이터 로딩 실패: {e}")
        return pd.DataFrame()



# 수출 예측 데이터를 불러오고 가공하는 함수 정의
def load_quarterly_export_forecast(db_info, root_hs_code, table_name='trade_forecast_by_month'):
    """
    특정 HS 코드를 기준으로 수출 예측 데이터를 분기별로 집계하고 변화율(pct_change)을 포함하여 반환하는 함수

    Parameters:
    - db_info (dict): DB 접속 정보
    - root_hs_code (str): 대상 HS 코드
    - table_name (str): DB 테이블명 (기본값: 'trade_forecast_by_month')

    Returns:
    - pd.DataFrame: 분기별 수출 예측 변화율 포함 데이터프레임
    """
    try:
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )

        query = f"""
            SELECT *
            FROM {table_name}
            WHERE root_hs_code = '{root_hs_code}'
            ORDER BY period
        """
        df = pd.read_sql(query, con=engine)
        df = df.drop_duplicates(subset=['period'])
        df['period'] = pd.to_datetime(df['period'])
        df['quarter'] = df['period'].dt.to_period('Q')
        quarterly_sum_df = df.groupby('quarter')['expDlr_forecast_12m'].sum().reset_index()
        quarterly_sum_df['quarter'] = quarterly_sum_df['quarter'].dt.to_timestamp()
        quarterly_sum_df['export_qoq_change'] = quarterly_sum_df['expDlr_forecast_12m'].pct_change(periods=1)
        quarterly_sum_df['export_yoy_change'] = quarterly_sum_df['expDlr_forecast_12m'].pct_change(periods=4)
        quarterly_sum_df['date_month'] = quarterly_sum_df['quarter'] + pd.offsets.QuarterEnd(0)
        quarterly_sum_df['root_hs_code'] = root_hs_code  # ✅ 구분자 추가
        return quarterly_sum_df

    except Exception as e:
        print(f"❌ 데이터 처리 실패: {e}")
        return pd.DataFrame()


# JSON 데이터를 불러오는 함수
def get_jsonparsed_data(url):
    context = ssl.create_default_context(cafile=certifi.where())
    with urlopen(url, context=context) as response:
        data = response.read().decode("utf-8")
        return json.loads(data)

# 주요 수치 변환용 컬럼 정의
compustat_is = ['report_date', 'ticker', 'period', 'sale', 'cogs', 'gp', 'xrd', 'xsga', 'idit', 'xint', 'dp', 'ebitda',
                'xopr', 'opiti', 'opir', 'pi', 'pir', 'txt', 'ni', 'nir', 'eps', 'epsdi', 'shrout', 'shroutdi']

## JSON 데이터를 불러오는 함수
def get_jsonparsed_data(url):
    context = ssl.create_default_context(cafile=certifi.where())
    with urlopen(url, context=context) as response:
        data = response.read().decode("utf-8")
        return json.loads(data)

# 주요 수치 변환용 컬럼 정의
compustat_is = ['report_date', 'ticker', 'period', 'sale', 'cogs', 'gp', 'xrd', 'xsga', 'idit', 'xint', 'dp', 'ebitda',
                'xopr', 'opiti', 'opir', 'pi', 'pir', 'txt', 'ni', 'nir', 'eps', 'epsdi', 'shrout', 'shroutdi']

# 진단용 로그 포함 함수
def forecast_sales_by_export_growth(ticker_list, forecast_quarters, quarterly_export_df):
    is_list = []
    results = []

    for ticker in tqdm(ticker_list, desc="Processing Tickers"):
        try:
            print(f"\n🔍 Processing {ticker}...")
            url = ("https://financialmodelingprep.com/api/v3/income-statement/{}?period=quarter&apikey=hT0gAk87j9xZx4PlBApvBqfVL5IahvgV".format(ticker))
            fs_raw = get_jsonparsed_data(url)
            print(f"✅ API 응답 수: {len(fs_raw)}")

            temp_df = pd.DataFrame(fs_raw)
            if temp_df.empty:
                print("❌ API 결과가 비어 있음.")
                continue

            is_df = temp_df[['date', 'symbol','period', 'revenue', 'costOfRevenue', 'grossProfit',
                             'researchAndDevelopmentExpenses', 'sellingGeneralAndAdministrativeExpenses',
                             'interestIncome', 'interestExpense', 'depreciationAndAmortization', 'ebitda',
                             'operatingExpenses', 'operatingIncome', 'operatingIncomeRatio', 'incomeBeforeTax',
                             'incomeBeforeTaxRatio','incomeTaxExpense', 'netIncome', 'netIncomeRatio', 'eps',
                             'epsdiluted', 'weightedAverageShsOut','weightedAverageShsOutDil']]
            is_df.columns = compustat_is
        except Exception as e:
            print(f"❌ 예외 발생: {e}")
            continue

        is_df_sorted = is_df.sort_values(by='report_date')
        is_df_sorted['report_date'] = pd.to_datetime(is_df_sorted['report_date'], errors='coerce')
        is_df_sorted['date_month'] = is_df_sorted['report_date'].dt.to_period('M').astype(str)
        is_df_sorted['date'] = pd.to_datetime(is_df_sorted['date_month']) + pd.offsets.MonthEnd(0)
        
        # 오늘 날짜 기준 2달 전 말일을 계산
        end_date = pd.Timestamp.today() - pd.DateOffset(months=2)
        end_date = end_date + pd.offsets.MonthEnd(0)
        
        # 날짜 범위 생성
        dates_list = pd.date_range('2004-01-31', end_date, freq='ME')
        date_df = pd.DataFrame(dates_list, columns=['date'])
        temp_is = pd.merge(date_df, is_df_sorted, on='date', how='left').ffill()

        target_data = temp_is[['date', 'date_month', 'ticker', 'period', 'sale', 'gp', 'ni']].copy()
        target_data_drop = target_data.drop_duplicates(subset=['date_month', 'period'], keep='last').copy()
        target_data_drop['date_month'] = pd.to_datetime(target_data_drop['date_month'])
        target_data_drop = target_data_drop.sort_values(['ticker', 'date_month']).reset_index(drop=True)

        for col in ['sale', 'gp', 'ni']:
            target_data_drop[f'{col}_yoy'] = target_data_drop.groupby('ticker')[col].transform(lambda x: x.pct_change(4))

        # 날짜 NaT 제거
        quarterly_export_df['date_month'] = pd.to_datetime(quarterly_export_df['date_month'])
        target_data_drop['date_month'] = pd.to_datetime(target_data_drop['date_month'])
        target_data_drop = target_data_drop[target_data_drop['date_month'].notna()].copy()
        quarterly_export_df = quarterly_export_df[quarterly_export_df['date_month'].notna()].copy()

        # 날짜 정렬
        target_data_drop = target_data_drop.sort_values('date_month')
        quarterly_export_df = quarterly_export_df.sort_values('date_month')

        print("📌 최근 매출 데이터:")
        print(target_data_drop[['date_month', 'sale']].tail())

        regression_table = pd.merge_asof(
            target_data_drop,
            quarterly_export_df[['date_month', 'export_yoy_change']],
            on='date_month',
            direction='nearest'
        )

        reg_df = regression_table[['export_yoy_change', 'sale_yoy']].copy()
        reg_df = reg_df.replace([np.inf, -np.inf], np.nan).dropna()

        print(f"📊 회귀분석 데이터 크기: {len(reg_df)}")
        if len(reg_df) < 5:
            print("⚠️ 유효한 회귀 데이터 부족. 건너뜀.")
            continue

        X = sm.add_constant(reg_df['export_yoy_change'])
        Y = reg_df['sale_yoy']
        model = sm.OLS(Y, X).fit()

        intercept = model.params['const']
        slope = model.params['export_yoy_change']
        p_value = model.pvalues['export_yoy_change']

        if target_data_drop['sale'].notna().sum() == 0:
            print("⚠️ 최근 매출 데이터 없음. 건너뜀.")
            continue

        last_sale = target_data_drop[target_data_drop['sale'].notna()].iloc[-1]['sale']
        current_sale = last_sale

        for f_date in forecast_quarters:
            export_yoy = quarterly_export_df[quarterly_export_df['date_month'] == pd.to_datetime(f_date)]['export_yoy_change']
            if export_yoy.empty:
                print(f"⛔ export_yoy_change 데이터 없음: {f_date}")
                continue

            export_yoy_val = export_yoy.values[0]
            predicted_growth = intercept + slope * export_yoy_val
            predicted_sale = current_sale * (1 + predicted_growth)

            results.append({
                'ticker': ticker,
                'forecated_salse': predicted_sale,
                'forecated_growth': predicted_growth,
                'slop': slope,
                'intercept': intercept,
                'p_value': p_value,
                'date_month': f_date, 
                'last_real_sale': last_sale 
            })

            current_sale = predicted_sale

        print(f"✅ {ticker} 예측 결과 {len(results)}건 누적")

    result_df = pd.DataFrame(results)
    
    # result_df_sum = result_df.groupby('ticker').apply(lambda df: df['forecated_salse'].sum() + df['last_real_sale'].iloc[0])
    
    return result_df


In [17]:
## 월별 추출입 데이터를 추출해 낸다 

# db_info = {
#     'host': '192.168.0.230',
#     'port': 3307,
#     'user': 'stox7412',
#     'password': 'Apt106503!~',
#     'database': 'investar'
# }
# 
# df_monthly = load_monthly_export_forecast(db_info)
# print(df_monthly.head())
# hscode_list = df_monthly['root_hs_code'].unique().tolist()
# pd.DataFrame(hscode_list).to_csv('korea_hscode_list.csv', index=False)

      period root_hs_code  final_expDlr_yoy  expDlr_forecast_12m
0 2007-01-01       121120               NaN                  NaN
1 2007-02-01       121120               NaN                  NaN
2 2007-03-01       121120               NaN                  NaN
3 2007-04-01       121120               NaN                  NaN
4 2007-05-01       121120               NaN                  NaN


In [13]:
# 결합 테스트 (함수 실행 예시)
quarterly_export_all_df = load_quarterly_export_forecast(
    db_info={
        'host': '192.168.0.230',
        'port': 3307,
        'user': 'stox7412',
        'password': 'Apt106503!~',
        'database': 'investar'
    },
    root_hs_code = '854232'
)

In [42]:
ticker_list = ['MU', 'AMAT', 'AVGO', 'LRCX', 'KLAC', 'ASML', 'COHR', 'UCTT', 'ICHR', 'ONTO', 'FORM', 'TER']

forecast_quarters = ['2025-06-30', '2025-09-30', '2026-12-31', '2026-03-31']

forecated_result = forecast_sales_by_export_growth(ticker_list, forecast_quarters, quarterly_export_df)

forecast_sum = (
    forecated_result.groupby('ticker', group_keys=False)
    .apply(lambda df: df['forecated_salse'].sum() + df['last_real_sale'].iloc[0])
)

Processing Tickers:   0%|          | 0/12 [00:00<?, ?it/s]


🔍 Processing MU...


Processing Tickers:   8%|▊         | 1/12 [00:01<00:17,  1.58s/it]

✅ API 응답 수: 159
📌 최근 매출 데이터:
   date_month          sale
80 2024-02-01  5.824000e+09
81 2024-05-01  6.811000e+09
82 2024-08-01  7.750000e+09
83 2024-11-01  8.709000e+09
84 2025-02-01  8.053000e+09
📊 회귀분석 데이터 크기: 65
⛔ export_yoy_change 데이터 없음: 2026-12-31
✅ MU 예측 결과 3건 누적

🔍 Processing AMAT...


Processing Tickers:  17%|█▋        | 2/12 [00:03<00:15,  1.54s/it]

...

Processing Tickers:  92%|█████████▏| 11/12 [00:15<00:01,  1.33s/it]

✅ API 응답 수: 89
📌 최근 매출 데이터:
   date_month         sale
80 2024-03-01  168725000.0
81 2024-06-01  197474000.0
82 2024-09-01  207917000.0
83 2024-12-01  189483000.0
84 2025-03-01  171356000.0
📊 회귀분석 데이터 크기: 65
⛔ export_yoy_change 데이터 없음: 2026-12-31
✅ FORM 예측 결과 33건 누적

🔍 Processing TER...


Processing Tickers: 100%|██████████| 12/12 [00:16<00:00,  1.39s/it]

✅ API 응답 수: 159
📌 최근 매출 데이터:
   date_month         sale
80 2024-03-01  597539000.0
81 2024-06-01  729879000.0
82 2024-09-01  737298000.0
83 2024-12-01  752884000.0
84 2025-03-01  685680000.0
📊 회귀분석 데이터 크기: 65
⛔ export_yoy_change 데이터 없음: 2026-12-31
✅ TER 예측 결과 36건 누적



C:\Users\82108\AppData\Local\Temp\ipykernel_4236\2116331126.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  forecated_result.groupby('ticker', group_keys=False)
